In [1]:
import json
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

from google.cloud import storage


In [2]:
BUCKET = "rjr-portal-da-transparencia"
YEAR = 2020

In [14]:
def load_json(name):
    bucket = storage.Client().get_bucket(BUCKET)
    blob = bucket.blob(f'aux/{name}.json')
    maps = json.loads(blob.download_as_string())
    return maps


def mapping(df, map_, column, type_return):
    map_func = udf(lambda key: map_.get(str(key)), type_return())
    df = df.withColumn(column, map_func(col(column)))
    return df

def string_to_date(df, column, year):
    if year > 2014:
        pattern = '%d/%m/%Y'
    else:
        pattern = "%d%b%Y:%H:%M:%S"
    map_func =  udf (lambda date: datetime.strptime(date, pattern) 
                     if type(date) == str 
                     else None, DateType())
    df = df.withColumn(column, map_func(col(column)))
    return df

In [4]:
spark = SparkSession.builder.appName("censo").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [22]:
schema = load_json("schemas/docentes_schema")
schema = StructType.fromJson(schema)
# except:
#     schema = StructType.fromJson(json.loads(schema))

In [11]:
schema = load_json("escolas_schema")
schema = StructType.fromJson(schema)
df = spark \
    .read \
    .options(header=True, delimiter="|", encoding="utf8") \
    .schema(schema=schema) \
    .csv(f"gs://{BUCKET}/landing_zone/censo-escolar/{YEAR}/escolas.csv" )

In [12]:
maps = load_json("maps")
string_columns = [column for column in df.columns 
                  if column.startswith("TP") or column.startswith("CO")]

for column in string_columns:
    if column in maps:
        df = mapping(df, maps[column], column, StringType)
    else:
        print(column)

CO_ENTIDADE
CO_DISTRITO
CO_ESCOLA_SEDE_VINCULADA


In [57]:
bool_columns = [column for column in df.columns 
                  if column.startswith("IN")]

mapping_bool = {
    "0": False,
    "1": True
}

for column in bool_columns:
    df = mapping(df, mapping_bool, column, BooleanType)

In [58]:
df = string_to_date(df, "DT_ANO_LETIVO_INICIO", YEAR)
df = string_to_date(df, "DT_ANO_LETIVO_TERMINO", YEAR)

In [61]:
df

NU_ANO_CENSO,CO_ENTIDADE,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,IN_AGUA_POCO_ARTESIANO,IN_AGUA_CACIMBA,IN_AGUA_FONTE_RIO,IN_AGUA_INEXISTENTE,IN_ENERGIA_REDE_PUBLICA,IN_ENERGIA_GERADOR,IN_ENERGIA_OUTROS,IN_ENERGIA_INEXISTENTE,IN_ESGOTO_REDE_PUBLICA,IN_ESGOTO_FOSSA,IN_ESGOTO_INEXISTENTE,IN_LIXO_COLETA_PERIODICA,IN_LIXO_QUEIMA,IN_LIXO_JOGA_OUTRA_AREA,IN_LIXO_RECICLA,IN_LIXO_ENTERRA,IN_LIXO_OUTROS,IN_SALA_DIRETORIA,IN_SALA_PROFESSOR,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_COBERTA,IN_QUADRA_ESPORTES_DESCOBERTA,IN_QUADRA_ESPORTES,IN_COZINHA,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BIBLIOTECA_SALA_LEITURA,IN_PARQUE_INFANTIL,IN_BERCARIO,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_DENTRO_PREDIO,IN_BANHEIRO_EI,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,IN_SECRETARIA,IN_BANHEIRO_CHUVEIRO,IN_REFEITORIO,IN_DESPENSA,IN_ALMOXARIFADO,IN_AUDITORIO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_ALOJAM_ALUNO,IN_ALOJAM_PROFESSOR,IN_AREA_VERDE,IN_LAVANDERIA,IN_DEPENDENCIAS_OUTRAS,NU_SALAS_EXISTENTES,NU_SALAS_UTILIZADAS,IN_EQUIP_TV,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_DVD,IN_EQUIP_PARABOLICA,IN_EQUIP_COPIADORA,IN_EQUIP_RETROPROJETOR,IN_EQUIP_IMPRESSORA,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_SOM,IN_EQUIP_MULTIMIDIA,IN_EQUIP_FAX,IN_EQUIP_FOTO,IN_COMPUTADOR,NU_EQUIP_TV,NU_EQUIP_VIDEOCASSETE,NU_EQUIP_DVD,NU_EQUIP_PARABOLICA,NU_EQUIP_COPIADORA,NU_EQUIP_RETROPROJETOR,NU_EQUIP_IMPRESSORA,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_SOM,NU_EQUIP_MULTIMIDIA,NU_EQUIP_FAX,NU_EQUIP_FOTO,NU_COMPUTADOR,NU_COMP_ADMINISTRATIVO,NU_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,NU_FUNCIONARIOS,IN_ALIMENTACAO,TP_AEE,TP_ATIVIDADE_COMPLEMENTAR,IN_FUNDAMENTAL_CICLOS,TP_LOCALIZACAO_DIFERENCIADA,IN_MATERIAL_ESP_QUILOMBOLA,IN_MATERIAL_ESP_INDIGENA,IN_MATERIAL_ESP_NAO_UTILIZA,IN_EDUCACAO_INDIGENA,TP_INDIGENA_LINGUA,CO_LINGUA_INDIGENA,IN_BRASIL_ALFABETIZADO,IN_FINAL_SEMANA,IN_FORMACAO_ALTERNANCIA,IN_MEDIACAO_PRESENCIAL,IN_MEDIACAO_SEMIPRESENCIAL,IN_MEDIACAO_EAD,IN_ESPECIAL_EXCLUSIVA,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF
2020,11000023,EEEE ABNAEL MACHA...,00009,Em Atividade,2020-02-06,2020-12-15,Norte,Madeira-Guaporé,Porto Velho,Rondônia,Porto Velho,110020505,Estadual,Urbana,null,true,null,false,false,null,null,null,null,null,null,null,null,null,null,false,null,null,null,true,Alugado,false,false,false,false,null,false,false,false,true,false,true,false,false,false,true,false,false,false,true,false,false,false,false,true,false,false,false,false,false,false,false,true,true,false,true,true,false,true,true,false,true,true,true,true,false,false,false,true,false,false,false,false,false,false,0,1,false,false,false,true,true,true,true,false,true,true,false,false,false,0,0,0,0,0,0,0,0,1,7,0,7,7,0,0,true,true,